# 아산병원 상세페이지 1278건 크롤링
`amc_disease_links.csv`에 담긴 링크를 기반으로
각 질환의 **질환명 / 증상 / 진료과** 정보를 수집하여 `amc_disease_1278pages.csv`로 저장합니다.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# 링크 CSV 파일 불러오기
df_links = pd.read_csv("amc_disease_links.csv")
links = df_links["상세페이지링크"].tolist()
print(f"총 링크 수: {len(links)}")

총 링크 수: 1278


In [3]:
def get_symptom_text(soup):
    # 1차 시도: 본문 내부 dt -> dd
    dts = soup.select("div.contDescription dt")
    for dt in dts:
        if "증상" in dt.get_text(strip=True):
            dd = dt.find_next_sibling("dd")
            if dd:
                ps = dd.find_all("p")
                if ps:
                    return " ".join(
                        p.get_text(strip=True).replace("\n", " ") for p in ps
                    )
                else:
                    return dd.get_text(strip=True).replace("\n", " ")

    # 2차 시도: 썸네일 옆 요약 정보
    try:
        dd = soup.select_one(
            "#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(2)"
        )
        if dd:
            return " ".join(
                a.get_text(strip=True).replace("\n", " ") for a in dd.find_all("a")
            )
    except:
        pass

    return ""

In [4]:
def get_symptom_text(soup):
    # 1차 시도: 본문 내부 dt -> dd
    dts = soup.select('div.contDescription dt')
    for dt in dts:
        if "증상" in dt.get_text(strip=True):
            dd = dt.find_next_sibling("dd")
            if dd:
                ps = dd.find_all("p")
                if ps:
                    return " ".join(p.get_text(strip=True) for p in ps)
                else:
                    return dd.get_text(strip=True)

    # 2차 시도: 썸네일 옆 요약 정보
    try:
        dd = soup.select_one(
            "#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(2)"
        )
        if dd:
            return ", ".join(a.get_text(strip=True) for a in dd.find_all("a"))
    except:
        pass

    return ''

In [5]:
# def get_symptom_text(soup):
#     try:
#         # 정확한 위치의 dd:nth-child(6) 가져오기
#         dd = soup.select_one(
#             "#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(6)"
#         )
#         if dd:
#             # 우선 <div><p>...</p></div> 구조 탐색
#             div = dd.find("div")
#             if div:
#                 ps = div.find_all("p")
#                 if ps:
#                     return "\n".join(p.get_text(strip=True) for p in ps)
#                 else:
#                     return div.get_text(strip=True)

#             # <p> 태그가 dd 안에 직접 있는 경우
#             ps = dd.find_all("p")
#             if ps:
#                 return "\n".join(p.get_text(strip=True) for p in ps)

#             # <p>도 <div>도 없는 경우: dd 내부 전체 텍스트
#             return dd.get_text(strip=True)

#     except Exception as e:
#         print(f"[증상 추출 오류] {e}")

#     return ""

In [6]:
def get_dept_text(soup):
    dts = soup.select("div.otherRegionBox dt")
    for dt in dts:
        if "진료과" in dt.get_text(strip=True):
            dd = dt.find_next_sibling("dd")
            if dd:
                return ", ".join(a.get_text(strip=True) for a in dd.find_all("a"))
    return ""

In [7]:
def get_section_text(soup, title):
    try:
        dts = soup.select("div.contDescription dt")
        for dt in dts:
            if title in dt.get_text(strip=True):
                dd = dt.find_next_sibling("dd")
                if dd:
                    # 1. dd > div > p 구조
                    div = dd.find("div")
                    if div:
                        ps = div.find_all("p")
                        if ps:
                            return " ".join(
                                p.get_text(strip=True).replace("\n", " ") for p in ps
                            )
                        return div.get_text(strip=True).replace("\n", " ")

                    # 2. dd > p 구조
                    ps = dd.find_all("p")
                    if ps:
                        return " ".join(
                            p.get_text(strip=True).replace("\n", " ") for p in ps
                        )

                    # 3. fallback: dd 텍스트 전체
                    return dd.get_text(strip=True).replace("\n", " ")
    except Exception as e:
        print(f"[{title} 추출 오류] {e}")

    return ""

In [8]:
results = []
headers = {"User-Agent": "Mozilla/5.0"}

# 셀렉터
sel_title = "#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > strong"


for i, url in enumerate(links):
    try:
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # 병명
        title_tag = soup.select_one(sel_title)
        title = title_tag.get_text(strip=True) if title_tag else ""

        # 증상
        symptom = get_symptom_text(soup)

        # 진료과
        dept = get_dept_text(soup)

        definition = get_section_text(soup, "정의")
        cause = get_section_text(soup, "원인")
        diagnosis = get_section_text(soup, "진단")
        treatment = get_section_text(soup, "치료")

        results.append(
            {
                "병명": title,
                "정의": definition,
                "원인": cause,
                "증상": symptom,
                "진단": diagnosis,
                "치료": treatment,
                "진료과": dept,
            }
        )

        print(f"[{i+1}/{len(links)}] 완료: {title}")
        time.sleep(1)

    except Exception as e:
        print(f"[에러] {url} - {e}")
        continue

[1/1278] 완료: 18번 염색체 단완결실 증후군(18p monosomy)
[2/1278] 완료: 18번 염색체 장완결실 증후군 (18q monosomy)
[3/1278] 완료: 22번 염색체 장완 미세결실 증후군(CATCH22)
[4/1278] 완료: 4번 염색체 단완결실 증후군(Wolf-Hirschhorn syndrome)
[5/1278] 완료: ARC 증후군(Arthrogryposis, Renal dysfunction, Cholestasis, ARC syndrome)
[6/1278] 완료: Haw River 증후군(Dentatorubro-Pallidoluysian atrophy(DRPLA))
[7/1278] 완료: MERRF(Myoclonic Epilepsy with Ragged Red Fiber) 증후군(Myoclonic epilepsy associated with ragged red fibers(MERRF))
[8/1278] 완료: OTC 결핍증 (Ornithine-transcarbamylase deficiency)
[9/1278] 완료: PSA 상승(Elevated PSA)
[10/1278] 완료: RS바이러스 감염증(RS virus infection)
[11/1278] 완료: Rh 부적합 임신(Rh Incompatibility pregnancy)
[12/1278] 완료: VDT 증후군(Visual Display Terminal Syndrome)
[13/1278] 완료: WPW 증후군(Wolff-Parkinson-White syndrome)
[14/1278] 완료: XXX 증후군(XXX syndrome)
[15/1278] 완료: XYY 증후군(XYY syndrome)
[16/1278] 완료: 가부키 증후군(Kabuki Syndrome)
[17/1278] 완료: 가성근시(Pseudomyopia)
[18/1278] 완료: 가스 괴저(Gas gangrene)
[19/1278] 완료: 가습기 살균제 연관 폐질환(Humidifier disinfectant

KeyboardInterrupt: 

In [ ]:
# # CSV 저장
df = pd.DataFrame(results)
df.to_csv("amc_disease_more.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 저장 완료")